In [32]:
from transformer_lens import HookedTransformer
import torch as t
import einops
from transformer_lens import utils as tutils

from functools import partial
import random
from tqdm import tqdm


from sae_lens import SparseAutoencoder
from sae_lens.toolkit.pretrained_saes import get_gpt2_res_jb_saes

In [33]:
import plotly.graph_objects as go

def display_lists_as_table(*lists, names=[], title=""):
    # Check if all lists have the same length
    list_lengths = [len(lst) for lst in lists]
    if len(set(list_lengths)) != 1:
        raise ValueError("All lists must have the same length.")

    # Create the header values
    header_values = [f'Column {i+1}' for i in range(len(lists[0]))]

    # Create the data for the table
    data = [go.Table(
        header=dict(values=names),
        cells=dict(values=lists),
    )]

    # Create the layout
    layout = go.Layout(
        title=title
    )

    # Create the figure and display the table
    fig = go.Figure(data=data, layout=layout)
    fig.show()

# Example usage
list1 = [1, 2, 3, 4, 5]
list2 = [6, 7, 8, 9, 10]
list3 = [11, 12, 13, 14, 15]

display_lists_as_table(list1, list2, names=["Token", "Value"])
display_lists_as_table(list1, list2, list3, names=["Token", "Value", "extra"])

In [34]:
model = HookedTransformer.from_pretrained(
    "gpt2-small",
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    refactor_factored_attn_matrices=True,
)

/usr/local/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning:

`resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.



Loaded pretrained model gpt2-small into HookedTransformer


In [35]:
layer = 7 # pick a layer you want.

hook_point = tutils.get_act_name("resid_pre", layer)
saes, sparsities = get_gpt2_res_jb_saes(hook_point)

print(saes.keys())
sae = saes[hook_point]
sae = sae.to(model.W_E.device)

100%|██████████| 1/1 [00:01<00:00,  1.32s/it]

dict_keys(['blocks.7.hook_resid_pre'])


In [36]:
def top_acts_at_pos(text, pos=-1, silent=True, prepend_bos=True, n_top=10):
    logits, cache = model.run_with_cache(text, prepend_bos=prepend_bos)
    if pos is None:
        hidden_state = cache[hook_point][0, :, :]
    else:
        hidden_state = cache[hook_point][0, pos, :].unsqueeze(0)
    feature_acts = sae(hidden_state).feature_acts
    feature_acts = feature_acts.mean(dim=0)
    top_v, top_i = t.topk(feature_acts, n_top)
    return top_v, top_i

top_activations = top_acts_at_pos("This day was like", pos=-1)
print(top_activations)

print(top_activations[0][0].item())
print(top_activations[1][0].item())

metaphor_vector = sae.W_dec[top_activations[1][0].item(), :]
metaphor_activation = top_activations[0][0].item()

print(metaphor_vector)
print(metaphor_activation)

(tensor([34.4923, 13.2345, 11.3497,  9.1794,  6.1575,  4.6734,  4.3486,  4.2683,
         3.7980,  3.7679], device='mps:0', grad_fn=<TopkBackward0>), tensor([14093,  6430, 22331, 19144, 24450, 23221,  6243,  3102,  4745,  5380],
       device='mps:0'))
34.49229049682617
14093
tensor([-4.9949e-03, -1.1252e-02, -2.5019e-02, -1.7989e-02, -2.3345e-02,
        -1.7728e-02,  1.4336e-02, -4.0011e-02,  2.1098e-02,  2.7976e-02,
         1.4074e-02, -1.2264e-02, -2.1630e-02, -9.8924e-03,  7.5899e-02,
        -9.5182e-02, -1.5417e-02,  1.3840e-02, -2.4799e-02,  1.8506e-02,
        -5.5362e-02, -3.7139e-02,  2.3834e-02, -5.1518e-02, -1.7197e-02,
         4.9247e-02,  2.6180e-02, -2.4630e-03,  1.5591e-02, -8.1095e-03,
        -1.6908e-02, -4.0562e-02,  5.1226e-02, -2.3033e-02, -1.0005e-02,
         2.0077e-02, -3.8991e-02,  1.4347e-02,  4.8115e-03, -5.5510e-02,
        -3.3198e-02,  1.9290e-02, -6.7268e-03,  2.3226e-02, -4.9797e-02,
        -4.7200e-03,  6.8628e-02,  2.3221e-02, -6.3930e-02,  5.595

In [112]:
def patch_hook(
    resid,
    hook,
    c,
    steering_vector,
    activation_norm
):    
    print(resid.shape)
    resid[:, 3, :] += c * activation_norm * steering_vector
    return resid

target_tokens = [" like", " a"]
target_vocab_positions = model.tokenizer.encode(target_tokens)

print(target_vocab_positions)

# logits, cache = model.run_with_cache("It's beautiful here, like")
for i in tqdm(range(1)):
    text = "Seeing this view was"

    print(model.tokenizer.tokenize(text))

    n_samples = 2

    new_samples = []

    for i in range(n_samples):
        if i < 2:
            logits = model.run_with_hooks(text, fwd_hooks=[(
                "blocks.5.hook_resid_pre",
                partial(patch_hook, c=40, steering_vector=metaphor_vector, activation_norm=metaphor_activation)
            )])
        else:
            logits, cache = model.run_with_cache(text)

        # return index positions for all vocab elements ex: [2339, 64]
        indices = logits[0,-1].topk(logits[0,-1].shape[0]).indices.tolist()
        ranks = [indices.index(v) for v in target_vocab_positions]

        # print(model.tokenizer.batch_decode(logits[0,-1].topk(20).indices))

        top_logits = logits[0,-1].topk(20)

        logit_positions = list(range(1, 21))
        tokens = model.tokenizer.batch_decode(top_logits.indices)
        values = top_logits.values.tolist()
        display_lists_as_table(logit_positions, tokens, values, names=["Logit Position", "Token", "Value"], title=text + "...")

        for v, r in zip(target_tokens, ranks):
            print(f"{v}_rank: {r}")

        # print(logits[0][-1].topk(3))
        # text += model.tokenizer.decode(logits[0][-1].topk(1).indices[0])
        text += model.tokenizer.decode(random.choice(logits[0][-1].topk(11).indices))
        new_samples.append(text)
        print(text)
    print(text)



[588, 257]


  0%|          | 0/1 [00:00<?, ?it/s]

['Seeing', 'Ġthis', 'Ġview', 'Ġwas']
torch.Size([1, 5, 768])


 like_rank: 1
 a_rank: 7
Seeing this view was like
torch.Size([1, 6, 768])


100%|██████████| 1/1 [00:01<00:00,  1.69s/it]

 like_rank: 186
 a_rank: 2
Seeing this view was like having
Seeing this view was like having


# Track a particular logit on a forward pass

In [38]:
anger = model.tokenizer.encode("anger")
you = model.tokenizer.encode(" you")

print(you)
print(anger)

# get rank of the word "anger" in the vocab
anger_rank = logits[0,-1].topk(logits[0,-1].shape[0]).indices.tolist().index(anger[0])
you_rank = logits[0,-1].topk(logits[0,-1].shape[0]).indices.tolist().index(you[0])

print(anger_rank)
print(you_rank)

[345]
[2564]
31796
60


In [39]:
like = model.tokenizer.encode("like")

like_rank = logits[0,-1].topk(logits[0,-1].shape[0]).indices.tolist().index(like[0])

print(like_rank)

28904


In [40]:
# project out the logits at every position
W_U = model.W_U

# cache residual post
for layer in range(model.cfg.n_layers):
    print(f"layer {layer}")
    residual_stream_layer = cache[f"blocks.{layer}.hook_resid_pre"]

    ## multiplying the current residual stream by W_U to get predicted tokens at that point
    ## this is the same as ablating all later layers and getting the logits
    unembedded_vocab = einops.einsum(
        residual_stream_layer,
        W_U,
        "b n d, d e -> b n e"
    )

    # print(unembedded_layer.shape)
    top_logits = unembedded_vocab[0,-1].topk(5)
    print(model.tokenizer.batch_decode(unembedded_vocab[0,-1].topk(5).indices))
print("final output")
print(model.tokenizer.batch_decode(logits[0,-1].topk(5).indices))

layer 0
[' meant', ' destro', ' mathemat', ' arrang', ' corrid']
layer 1
[' meant', 'stress', ' wrongly', ' expressly', ' Pv']
layer 2
[' meant', ' spelling', ' wrongly', ' feeding', 'hift']
layer 3
[' precisely', ' meant', ' perfectly', ' everything', ' something']
layer 4
[' something', ' goodbye', ' precisely', 'hift', ' greatly']
layer 5
[' goodbye', ' something', ' saving', ' everything', ' perfectly']
layer 6
[' something', ' goodbye', ' precisely', ' perfectly', ' everything']
layer 7
[' something', ' goodbye', ' everything', ' nothing', ' anything']
layer 8
[' something', ' nothing', ' goodbye', ' everything', ' anything']
layer 9
[' something', ' nothing', ' sacrificing', ' everything', ' anything']
layer 10
[' something', ' sacrificing', ' nothing', ' saving', ' abandoning']
layer 11
[' something', ' sacrificing', ' abandoning', ' nothing', ' shutting']
final output
[' that', ' something', ' a', ' the', ' we']


In [41]:
# I want to be able to track specific tokens
# I want to be able to see the top activations.

top_logit = logits[0,-1].topk(1).indices.item()
print(top_logit)


326


In [42]:
# Ok we want some kind of metric that we can use from this stuff.
# When we make an intervention, how does it change the downstream activations? The logits?
# And specifically, how does it change particular logits that we are interested in?

# We can do this by looking at the activations of the token we are interested in, and then looking at the activations of the token we are interested in after the intervention.


In [113]:
import pandas as pd

# Create a sample dataset
data = [
    ["Test", 1.239],
    ["Example", 0.859],
    ["Python", 1.523],
    ["Notebook", 0.742],
    ["Table", 1.187]
]

# Create a pandas DataFrame
df = pd.DataFrame(data, columns=["Token", "Probability"])

# Define custom styles
styles = [
    {'selector': 'th', 'props': [('background-color', '#4CAF50'), 
                                 ('color', 'white'), 
                                 ('text-align', 'center'), 
                                 ('padding', '10px'),
                                 ('font-size', '14px'),
                                 ('border-bottom', '2px solid #ddd')]},
    {'selector': 'td', 'props': [('text-align', 'center'), 
                                 ('padding', '8px'),
                                 ('font-size', '13px'),
                                 ('border-bottom', '1px solid #ddd')]},
    {'selector': 'tr:nth-child(odd)', 'props': [('background-color', '#f2f2f2')]},
    {'selector': 'tr:nth-child(even)', 'props': [('background-color', 'white')]},
    {'selector': 'table', 'props': [('border-collapse', 'collapse'), 
                                    ('width', '50%'), 
                                    ('margin', '25px 0'),
                                    ('font-family', 'Arial, sans-serif')]}
]

# Apply the styles to the DataFrame
styled_df = df.style.set_table_styles(styles)

# Display the styled table
styled_df


,Token,Probability
0,Test,1.239000
1,Example,0.859000
2,Python,1.523000
3,Notebook,0.742000
4,Table,1.187000


In [123]:
import pandas as pd
from IPython.display import display

# Create a sample dataset
data = [
    ["Test", 1.239],
    ["Example", 0.859],
    ["Python", 1.523],
    ["Notebook", 0.742],
    ["Table", 1.187]
]

# Create a pandas DataFrame
df = pd.DataFrame(data, columns=["Token", "Probability"])

# Define custom styles
styles = [
    {'selector': 'th', 'props': [('background-color', '#dddddd'), 
                                 ('color', 'black'), 
                                 ('text-align', 'center'), 
                                 ('padding', '10px'),
                                 ('font-size', '16px'),
                                 ('border-bottom', '2px solid #ccc')]},
    {'selector': 'td', 'props': [('text-align', 'center'), 
                                 ('padding', '8px'),
                                 ('font-size', '14px'),
                                 ('border-bottom', '1px solid #ccc'),
                                 ('color', 'black')]},
    {'selector': 'tr:nth-child(odd)', 'props': [('background-color', '#f9f9f9')]},
    {'selector': 'tr:nth-child(even)', 'props': [('background-color', 'white')]},
    {'selector': 'table', 'props': [('border-collapse', 'collapse'), 
                                    ('width', '50%'), 
                                    ('margin', '25px 0'),
                                    ('font-family', 'Arial, sans-serif'),
                                    ('border-radius', '8px'),
                                    ('overflow', 'hidden')]},
    {'selector': 'caption', 'props': [('caption-side', 'top'),
                                      ('font-size', '18px'),
                                      ('font-weight', 'bold'),
                                      ('margin-bottom', '10px')]}
]

# Additional styles for hover effect and column width
hover_styles = [
    {'selector': 'tbody tr:hover', 'props': [('background-color', '#e0e0e0')]}
]

# Apply the styles to the DataFrame and add a title
styled_df = df.style.set_table_styles(styles).set_caption("Top Tokens")
styled_df = styled_df.set_table_styles(hover_styles, overwrite=False)
display(styled_df)



,Token,Probability
0,Test,1.239000
1,Example,0.859000
2,Python,1.523000
3,Notebook,0.742000
4,Table,1.187000


In [99]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

# Sample data for demonstration
data1 = pd.DataFrame({
    'A': [1, 2, 3],
    'B': [4, 5, 6],
    'C': [7, 8, 9]
})

data2 = pd.DataFrame({
    'A': [10, 20, 30],
    'B': [40, 50, 60],
    'C': [70, 80, 90]
})

data3 = pd.DataFrame({
    'A': [100, 200, 300],
    'B': [400, 500, 600],
    'C': [700, 800, 900]
})

# Dictionary to map buttons to data
data_dict = {
    'Show Data 1': data1,
    'Show Data 2': data2,
    'Show Data 3': data3
}

# Variable to keep track of the currently selected button
selected_button = None
current_data = data1

# Function to update the table and button styles based on button clicked
def update_table(button):
    global selected_button, current_data
    # Reset the style of the previously selected button
    if selected_button:
        selected_button.style.button_color = None
    # Update the style of the currently selected button
    button.style.button_color = 'lightblue'
    selected_button = button
    current_data = data_dict[button.description]
    # Update the table
    table_widget.value = current_data.to_html()

# Function to add a new row
def add_row(_):
    global current_data
    new_row = pd.DataFrame({col: [input_widgets[col].value] for col in current_data.columns})
    current_data = pd.concat([current_data, new_row], ignore_index=True)
    table_widget.value = current_data.to_html()

# Create buttons
buttons = []
for key in data_dict.keys():
    button = widgets.Button(description=key)
    button.on_click(update_table)
    buttons.append(button)

# Display buttons in a horizontal box that wraps if necessary
buttons_box = widgets.HBox(buttons)

# Create an HTML widget to display the table
table_widget = widgets.HTML()

# Create input widgets for each column
input_widgets = {col: widgets.Text(description=col) for col in data1.columns}
input_widgets_box = widgets.VBox([input_widgets[col] for col in data1.columns])

# Create a button to add a new row
add_row_button = widgets.Button(description="Add Row")
add_row_button.on_click(add_row)

# Display the input widgets and add row button vertically
input_area = widgets.VBox([input_widgets_box, add_row_button])

# Display the buttons, input area, and table side by side
display(widgets.HBox([widgets.VBox([buttons_box, input_area]), table_widget]))

# Initialize the table with the first dataset and style the first button
buttons[0].style.button_color = 'lightblue'
selected_button = buttons[0]
table_widget.value = data1.to_html()
current_data = data1


In [102]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

# Sample data for demonstration
data1 = pd.DataFrame({
    'A': [1, 2, 3],
    'B': [4, 5, 6],
    'C': [7, 8, 9]
})

data2 = pd.DataFrame({
    'A': [10, 20, 30],
    'B': [40, 50, 60],
    'C': [70, 80, 90]
})

data3 = pd.DataFrame({
    'A': [100, 200, 300],
    'B': [400, 500, 600],
    'C': [700, 800, 900]
})

# Dictionary to map buttons to data
data_dict = {
    'Show Data 1': data1,
    'Show Data 2': data2,
    'Show Data 3': data3
}

# Variable to keep track of the currently selected button
selected_button = None
current_data = data1

# Function to update the table and button styles based on button clicked
def update_table(button):
    global selected_button, current_data
    # Reset the style of the previously selected button
    if selected_button:
        selected_button.style.button_color = None
    # Update the style of the currently selected button
    button.style.button_color = 'lightblue'
    selected_button = button
    current_data = data_dict[button.description]
    # Update the table
    update_table_html()

# Function to update the table HTML with colored new rows
def update_table_html():
    global current_data
    html = current_data.to_html(classes='table table-striped', index=False)
    for row in new_rows:
        row_html = pd.DataFrame([row]).to_html(header=False, index=False)
        html = html.replace('</tbody>', f'<tr style="background-color: lightyellow">{row_html[5:-14]}</tr></tbody>')
    table_widget.value = html

# Function to add a new row
def add_row(_):
    global current_data
    new_row = {col: input_widgets[col].value for col in current_data.columns}
    new_rows.append(new_row)
    for key in data_dict:
        new_row_df = pd.DataFrame([new_row])
        data_dict[key] = pd.concat([new_row_df, data_dict[key]], ignore_index=True)
    update_table_html()

# Initialize list to keep track of new rows
new_rows = []

# Create buttons
buttons = []
for key in data_dict.keys():
    button = widgets.Button(description=key)
    button.on_click(update_table)
    buttons.append(button)

# Display buttons in a horizontal box that wraps if necessary
buttons_box = widgets.HBox(buttons)

# Create an HTML widget to display the table
table_widget = widgets.HTML()

# Create input widgets for each column
input_widgets = {col: widgets.Text(description=col) for col in data1.columns}
input_widgets_box = widgets.VBox([input_widgets[col] for col in data1.columns])

# Create a button to add a new row
add_row_button = widgets.Button(description="Add Row")
add_row_button.on_click(add_row)

# Display the input widgets and add row button vertically
input_area = widgets.VBox([input_widgets_box, add_row_button])

# Display the buttons, input area, and table side by side
display(widgets.HBox([widgets.VBox([buttons_box, input_area]), table_widget]))

# Initialize the table with the first dataset and style the first button
buttons[0].style.button_color = 'lightblue'
selected_button = buttons[0]
update_table_html()


In [104]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

# Sample data for demonstration
data1 = pd.DataFrame({
    'A': [1, 2, 3],
    'B': [4, 5, 6],
    'C': [7, 8, 9]
})

data2 = pd.DataFrame({
    'A': [10, 20, 30],
    'B': [40, 50, 60],
    'C': [70, 80, 90]
})

data3 = pd.DataFrame({
    'A': [100, 200, 300],
    'B': [400, 500, 600],
    'C': [700, 800, 900]
})

# Dictionary to map buttons to data
data_dict = {
    'Show Data 1': data1,
    'Show Data 2': data2,
    'Show Data 3': data3
}

# Variable to keep track of the currently selected button
selected_button = None
current_data = data1

# Function to update the table and button styles based on button clicked
def update_table(button):
    global selected_button, current_data
    # Reset the style of the previously selected button
    if selected_button:
        selected_button.style.button_color = None
    # Update the style of the currently selected button
    button.style.button_color = 'lightblue'
    selected_button = button
    current_data = data_dict[button.description]
    # Update the table
    update_table_html()

# Function to update the table HTML with colored new rows
def update_table_html():
    global current_data
    combined_data = pd.concat([pd.DataFrame(new_rows), current_data], ignore_index=True)
    html = combined_data.to_html(classes='table table-striped', index=False)
    table_widget.value = html

# Function to add a new row
def add_row(_):
    new_row = {col: input_widgets[col].value for col in data1.columns}
    new_rows.append(new_row)
    update_table_html()

# Initialize list to keep track of new rows
new_rows = []

# Create buttons
buttons = []
for key in data_dict.keys():
    button = widgets.Button(description=key)
    button.on_click(update_table)
    buttons.append(button)

# Display buttons in a horizontal box that wraps if necessary
buttons_box = widgets.HBox(buttons)

# Create an HTML widget to display the table
table_widget = widgets.HTML()

# Create input widgets for each column
input_widgets = {col: widgets.Text(description=col) for col in data1.columns}
input_widgets_box = widgets.VBox([input_widgets[col] for col in data1.columns])

# Create a button to add a new row
add_row_button = widgets.Button(description="Add Row")
add_row_button.on_click(add_row)

# Display the input widgets and add row button vertically
input_area = widgets.VBox([input_widgets_box, add_row_button])

# Display the buttons, input area, and table side by side
display(widgets.HBox([widgets.VBox([buttons_box, table_widget]), input_area]))

# Initialize the table with the first dataset and style the first button
buttons[0].style.button_color = 'lightblue'
selected_button = buttons[0]
update_table_html()


In [110]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

# Sample data for demonstration
data = [
    pd.DataFrame({
        'A': [i*10 + 1, i*10 + 2, i*10 + 3],
        'B': [i*10 + 4, i*10 + 5, i*10 + 6],
        'C': [i*10 + 7, i*10 + 8, i*10 + 9]
    }) for i in range(10)
]

# Dictionary to map button descriptions to data
data_dict = {f'Show Data {i+1}': data[i] for i in range(len(data))}

# Variable to keep track of the currently selected button
selected_button = None
current_data = data[0]

# Function to update the table and button styles based on button clicked
def update_table(button):
    global selected_button, current_data
    # Reset the style of the previously selected button
    if selected_button:
        selected_button.style.button_color = None
    # Update the style of the currently selected button
    button.style.button_color = 'lightblue'
    selected_button = button
    current_data = data_dict[button.description]
    # Update the table
    update_table_html()

# Function to update the table HTML with colored new rows
def update_table_html():
    global current_data
    combined_data = pd.concat([pd.DataFrame(new_rows), current_data], ignore_index=True)
    html = combined_data.to_html(classes='table table-striped', index=False)
    table_widget.value = html

# Function to add a new row
def add_row(_):
    new_row = {col: input_widgets[col].value for col in current_data.columns}
    new_rows.append(new_row)
    update_table_html()

# Initialize list to keep track of new rows
new_rows = []

# Create buttons
buttons = []
for key in data_dict.keys():
    button = widgets.Button(description=key)
    button.on_click(update_table)
    button.layout.padding = '0px'
    button.layout.margin = '0px'
    button.style.button_color = 'white'
    button.layout.border = '1px solid lightgrey'
    buttons.append(button)

# Display buttons in a horizontal box that wraps if necessary
buttons_box = widgets.Box(buttons, layout=widgets.Layout(display='flex', flex_flow='row wrap'))

# Create an HTML widget to display the table
table_widget = widgets.HTML()

# Create input widgets for each column
input_widgets = {col: widgets.Text(description=col) for col in current_data.columns}
input_widgets_box = widgets.VBox([input_widgets[col] for col in current_data.columns])

# Create a button to add a new row
add_row_button = widgets.Button(description="Add Row")
add_row_button.on_click(add_row)

# Display the input widgets and add row button vertically
input_area = widgets.VBox([input_widgets_box, add_row_button])

# Display the buttons, input area, and table side by side
display(widgets.HBox([widgets.VBox([buttons_box, table_widget]), input_area]))

# Initialize the table with the first dataset and style the first button
buttons[0].style.button_color = 'lightblue'
selected_button = buttons[0]
update_table_html()


# Wdiget exploration

In [43]:
%pip install ipywidgets
%pip install IPython

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [44]:
import ipywidgets as widgets
from IPython.display import display

# Assume you have a list of text snippets and their associated arrays
snippets = ["hi", "bueno", "gracias", "mom"]
array1 = [[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12]]
array2 = [["a", "b", "c"], ["d", "e", "f"], ["g", "h", "i"], ["j", "k", "l"]]
array3 = [[True, False, True], [False, True, False], [True, True, False], [False, False, True]]

# Create a list of buttons for each text snippet
buttons = [widgets.Button(description=snippet, layout=widgets.Layout(width='auto', border='1px solid black', padding='0px', margin="0px")) for snippet in snippets]

# Remove the background color from the buttons
for button in buttons:
    button.style.button_color = 'transparent'

# Create a box to hold the buttons
button_box = widgets.HBox(buttons, layout=widgets.Layout(display='flex', flex_flow='row wrap', align_items='stretch'))

# Create an output widget to display the table
output = widgets.Output()

# Create a dropdown widget for selecting text options
text_options = ["Option 1", "Option 2", "Option 3"]
dropdown = widgets.Dropdown(options=text_options, description="Select text:")

# Create a submit button
submit_button = widgets.Button(description="Submit")

# Create a box to hold the dropdown and submit button
input_box = widgets.VBox([dropdown, submit_button])

# Create a box to hold the button_box and input_box side by side
main_box = widgets.HBox([button_box, input_box])

# Define a function to update the output and highlight the selected button
def on_button_click(button):
    index = snippets.index(button.description)
    data = [array1[index], array2[index], array3[index]]
    headers = ["Array 1", "Array 2", "Array 3"]
    output.clear_output()
    with output:
        display(widgets.HTML(
            '<table><tr>{}</tr>{}</table>'.format(
                ''.join('<th>{}</th>'.format(header) for header in headers),
                ''.join('<tr>{}</tr>'.format(''.join('<td>{}</td>'.format(str(item)) for item in row)) for row in zip(*data))
            )
        ))
    for btn in buttons:
        btn.style.button_color = 'lightblue' if btn == button else 'transparent'

# Define a function to handle the submit button click
def on_submit_click(button):
    selected_option = dropdown.value
    # Add the selected option to the snippets list
    snippets.append(selected_option)
    # Add dummy data to the arrays (replace with your own logic)
    array1.append([1, 2, 3])
    array2.append(["a", "b", "c"])
    array3.append([True, False, True])
    # Create a new button for the added snippet
    new_button = widgets.Button(description=selected_option, layout=widgets.Layout(width='auto', border='1px solid black', padding='0px', margin="0px"))
    new_button.style.button_color = 'transparent'
    new_button.on_click(lambda btn: on_button_click(btn))
    buttons.append(new_button)
    button_box.children = tuple(buttons)

# Attach the on_button_click function to each button
for button in buttons:
    button.on_click(lambda btn: on_button_click(btn))

# Attach the on_submit_click function to the submit button
submit_button.on_click(on_submit_click)

# Display the main box and output widgets
display(main_box)
display(output)

Output()

ValueError: 'Option 2' is not in list

In [45]:
import ipywidgets as widgets
from IPython.display import display

# Assume you have a list of text snippets and their associated arrays
snippets = ["hi", "bueno", "gracias", "mom"]
array1 = [[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12]]
array2 = [["a", "b", "c"], ["d", "e", "f"], ["g", "h", "i"], ["j", "k", "l"]]
array3 = [[True, False, True], [False, True, False], [True, True, False], [False, False, True]]
colors = ['lightblue', 'lightgreen', 'lightyellow', 'lightpink']

# Create a list of buttons for each text snippet
buttons = [widgets.Button(description=snippet, layout=widgets.Layout(width='auto', border='1px solid black', padding='0px', margin="0px")) for snippet in snippets]

# Remove the background color from the buttons
for button, color in zip(buttons, colors):
    button.style.button_color = color

# Create a box to hold the buttons
button_box = widgets.HBox(buttons, layout=widgets.Layout(display='flex', flex_flow='row wrap', align_items='stretch'))

# Create an output widget to display the table
output = widgets.Output()

# Create input widgets for adding new data
text_input = widgets.Text(description="Text:")
color_picker = widgets.ColorPicker(description="Color:", value='#ffffff')
add_button = widgets.Button(description="+")

# Create a box to hold the input widgets
input_box = widgets.HBox([text_input, color_picker, add_button])

# Create a box to hold the button_box and input_box
main_box = widgets.VBox([button_box, input_box, output])

# Define a function to update the output and highlight the selected button
def on_button_click(button):
    index = snippets.index(button.description)
    data = [array1[index], array2[index], array3[index]]
    headers = ["Array 1", "Array 2", "Array 3"]
    table_rows = [
        '<tr style="background-color: {};">{}</tr>'.format(
            colors[index],
            ''.join('<td>{}</td>'.format(str(item)) for item in row)
        )
        for row in zip(*data)
    ]
    table_html = '''
        <table>
            <tr><th colspan="3">Data Types</th></tr>
            <tr><td>int</td><td>str</td><td>bool</td></tr>
            <tr><td colspan="3"><button onclick="addRow()">+</button></td></tr>
            <tr>{}</tr>
            {}
        </table>
    '''.format(
        ''.join('<th>{}</th>'.format(header) for header in headers),
        ''.join(table_rows)
    )
    output.clear_output()
    with output:
        display(widgets.HTML(table_html))

# Define a function to handle adding a new row
def on_add_click(button):
    text = text_input.value.strip()
    if text:
        color = color_picker.value
        snippets.append(text)
        array1.append([1, 2, 3])  # Replace with your own logic
        array2.append(["a", "b", "c"])  # Replace with your own logic
        array3.append([True, False, True])  # Replace with your own logic
        colors.append(color)
        new_button = widgets.Button(description=text, layout=widgets.Layout(width='auto', border='1px solid black', padding='0px', margin="0px"))
        new_button.style.button_color = color
        new_button.on_click(lambda btn: on_button_click(btn))
        buttons.append(new_button)
        button_box.children = tuple(buttons)
        text_input.value = ""

# Attach the on_button_click function to each button
for button in buttons:
    button.on_click(lambda btn: on_button_click(btn))

# Attach the on_add_click function to the add button
add_button.on_click(on_add_click)

# Display the main box
display(main_box)

TypeError: update_table() takes 0 positional arguments but 1 was given

In [67]:
import ipywidgets as widgets
from IPython.display import display
import uuid

# Assume you have a list of text snippets and their associated arrays
selected_id = 0
snippets = ["hi", "bueno", "gracias", "mom"]
snippet_ids = [str(uuid.uuid4()) for _ in range(len(snippets))]
array1 = [[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12]]
array2 = [["a", "b", "c"], ["d", "e", "f"], ["g", "h", "i"], ["j", "k", "l"]]
array3 = [[True, False, True], [False, True, False], [True, True, False], [False, False, True]]
colors = ['lightblue', 'lightgreen', 'lightyellow', 'lightpink']

# Create a list of buttons for each text snippet
buttons = [widgets.Button(description=snippet, layout=widgets.Layout(width='auto', border='1px solid black', padding='0px', margin="0px")) for snippet in snippets]

# Create a box to hold the buttons
button_box = widgets.HBox(buttons, layout=widgets.Layout(display='flex', flex_flow='row wrap', align_items='stretch'))

# Create an output widget to display the table
output = widgets.Output()

# Create input widgets for adding new data
text_input = widgets.Text(description="Text:", margin="0px")
color_picker = widgets.ColorPicker(description="Color:", value='#ffffff', margin="0px")
add_button = widgets.Button(description="+")

# Create a box to hold the input widgets
input_box = widgets.VBox([text_input, color_picker, add_button])

# Create a box to hold the button_box and input_box
main_box = widgets.VBox([button_box, input_box, output])

# Define a function to update the table
def update_table():
    print(selected_id)
    headers = ["Array 1", "Array 2", "Array 3"]
    table_rows = []
    # for i in range(len(snippets)):
    data = [array1[selected_id], array2[selected_id], array3[selected_id]]
    table_rows.extend(
        '<tr style="background-color: {};">{}</tr>'.format(
            colors[selected_id],
            ''.join('<td>{}</td>'.format(str(item)) for item in row)
        )
        for row in zip(*data)
    )
    table_html = '''
        <table>
            <tr>{}</tr>
            {}
        </table>
    '''.format(
        ''.join('<th>{}</th>'.format(header) for header in headers),
        ''.join(table_rows)
    )
    output.clear_output(wait=True)
    with output:
        display(widgets.HTML(table_html))

# Define a function to handle adding a new row
def on_add_click(button):
    text = text_input.value.strip()
    if text:
        color = color_picker.value
        snippet_id = str(uuid.uuid4())
        snippet_ids.append(snippet_id)
        snippets.append(text)
        array1.append([1, 2, 3])  # Replace with your own logic
        array2.append([text])
        array3.append([True, False, True])  # Replace with your own logic
        colors.append(color)
        update_table()
        text_input.value = ""



# Attach the on_add_click function to the add button
add_button.on_click(on_add_click)

# Define a function to update the output and highlight the selected button
def on_button_click(button):
    # index = snippets.index(button.description)
    # data = [array1[index], array2[index], array3[index]]
    # headers = ["Array 1", "Array 2", "Array 3"]
    # output.clear_output()
    # with output:
    #     display(widgets.HTML(
    #         '<table><tr>{}</tr>{}</table>'.format(
    #             ''.join('<th>{}</th>'.format(header) for header in headers),
    #             ''.join('<tr>{}</tr>'.format(''.join('<td>{}</td>'.format(str(item)) for item in row)) for row in zip(*data))
    #         )
    #     ))
    
    for i, btn in enumerate(buttons):
        btn.style.button_color = 'lightblue' if btn == button else 'transparent'
        selected_id = i



# Attach the on_button_click function to each button
for button in buttons:
    button.on_click(lambda btn: on_button_click(btn))


# Display the main box and initial table
display(main_box)
update_table()

0


0


In [53]:
import ipywidgets as widgets
from IPython.display import display
import uuid

# Assume you have a list of text snippets and their associated arrays
snippets = ["hi", "bueno", "gracias", "mom"]
snippet_ids = [str(uuid.uuid4()) for _ in range(len(snippets))]
array1 = [[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12]]
array2 = [["a", "b", "c"], ["d", "e", "f"], ["g", "h", "i"], ["j", "k", "l"]]
array3 = [[True, False, True], [False, True, False], [True, True, False], [False, False, True]]
colors = ['lightgray', 'darkgray', 'lightgray', 'darkgray']

# Create a list to store the data added from the interface
added_data = []

# Create a list of buttons for each text snippet
buttons = [widgets.Button(description=snippet, layout=widgets.Layout(width='auto', border='1px solid black', padding='0px', margin="0px")) for snippet in snippets]

# Create a box to hold the buttons
button_box = widgets.HBox(buttons, layout=widgets.Layout(display='flex', flex_flow='row wrap', align_items='stretch'))

# Create an output widget to display the table
output = widgets.Output()

# Create input widgets for adding new data
text_input = widgets.Text(description="Text:")
color_picker = widgets.ColorPicker(description="Color:", value='#ffffff')
add_button = widgets.Button(description="+")

# Create a box to hold the input widgets
input_box = widgets.HBox([text_input, color_picker, add_button])

# Create a box to hold the button_box and input_box
main_box = widgets.VBox([button_box, input_box, output])

# Define a function to update the table based on the selected text snippet
def update_table(snippet_index):
    headers = ["Array 1", "Array 2", "Array 3"]
    table_rows = []

    # Add the data from the interface
    for data in added_data:
        table_rows.extend(
            '<tr style="background-color: {};">{}</tr>'.format(
                data['color'],
                ''.join('<td>{}</td>'.format(str(item)) for item in row)
            )
            for row in zip(*data['arrays'])
        )

    # Add the data for the selected text snippet
    data = [array1[snippet_index], array2[snippet_index], array3[snippet_index]]
    table_rows.extend(
        '<tr style="background-color: {};">{}</tr>'.format(
            colors[snippet_index],
            ''.join('<td>{}</td>'.format(str(item)) for item in row)
        )
        for row in zip(*data)
    )

    table_html = '''
        <table>
            <tr><th colspan="3">Data Types</th></tr>
            <tr><td>int</td><td>str</td><td>bool</td></tr>
            <tr><td colspan="3"><button onclick="addRow()">+</button></td></tr>
            <tr>{}</tr>
            {}
        </table>
    '''.format(
        ''.join('<th>{}</th>'.format(header) for header in headers),
        ''.join(table_rows)
    )
    output.clear_output(wait=True)
    with output:
        display(widgets.HTML(table_html))

# Define a function to handle button clicks
def on_button_click(button):
    snippet_index = snippets.index(button.description)
    update_table(snippet_index)

# Attach the on_button_click function to each button
for button in buttons:
    button.on_click(lambda btn: on_button_click(btn))

# Define a function to handle adding a new row
def on_add_click(button):
    text = text_input.value.strip()
    if text:
        color = color_picker.value
        snippet_id = str(uuid.uuid4())
        snippet_ids.append(snippet_id)
        snippets.append(text)
        array1_data = [1, 2, 3]  # Replace with your own logic
        array2_data = [text]
        array3_data = [True, False, True]  # Replace with your own logic
        added_data.append({
            'snippet_id': snippet_id,
            'arrays': [array1_data, array2_data, array3_data],
            'color': color
        })
        update_table(len(snippets) - 1)
        text_input.value = ""

# Attach the on_add_click function to the add button
add_button.on_click(on_add_click)

# Display the main box and initial table
display(main_box)
update_table(0)

IndexError: list index out of range

In [54]:
import ipywidgets as widgets
from IPython.display import display
import uuid

# Assume you have a list of text snippets and their associated arrays
snippets = ["hi", "bueno", "gracias", "mom"]
snippet_ids = [str(uuid.uuid4()) for _ in range(len(snippets))]
array1 = [[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12]]
array2 = [["a", "b", "c"], ["d", "e", "f"], ["g", "h", "i"], ["j", "k", "l"]]
array3 = [[True, False, True], [False, True, False], [True, True, False], [False, False, True]]
colors = ['lightgray', 'darkgray', 'lightgray', 'darkgray']

# Create a list to store the data added from the interface
added_data = []

# Create a list of buttons for each text snippet
buttons = [widgets.Button(description=snippet, layout=widgets.Layout(width='auto', border='1px solid black', padding='0px', margin="0px")) for snippet in snippets]

# Create a box to hold the buttons
button_box = widgets.HBox(buttons, layout=widgets.Layout(display='flex', flex_flow='row wrap', align_items='stretch'))

# Create an output widget to display the table
output = widgets.Output()

# Create input widgets for adding new data
text_input = widgets.Text(description="Text:")
color_picker = widgets.ColorPicker(description="Color:", value='#ffffff')
add_button = widgets.Button(description="+")

# Create a box to hold the input widgets
input_box = widgets.HBox([text_input, color_picker, add_button])

# Create a box to hold the button_box and input_box
main_box = widgets.VBox([button_box, input_box, output])

# Define a function to update the table based on the selected text snippet
def update_table(snippet_index):
    headers = ["Array 1", "Array 2", "Array 3"]
    table_rows = []

    # Add the data from the interface
    for data in added_data:
        table_rows.extend(
            '<tr style="background-color: {};">{}</tr>'.format(
                data['color'],
                ''.join('<td>{}</td>'.format(str(item)) for item in row)
            )
            for row in zip(*data['arrays'])
        )

    # Add the data for the selected text snippet if the index is valid
    if 0 <= snippet_index < len(snippets):
        data = [array1[snippet_index], array2[snippet_index], array3[snippet_index]]
        table_rows.extend(
            '<tr style="background-color: {};">{}</tr>'.format(
                colors[snippet_index],
                ''.join('<td>{}</td>'.format(str(item)) for item in row)
            )
            for row in zip(*data)
        )

    table_html = '''
        <table>
            <tr><th colspan="3">Data Types</th></tr>
            <tr><td>int</td><td>str</td><td>bool</td></tr>
            <tr><td colspan="3"><button onclick="addRow()">+</button></td></tr>
            <tr>{}</tr>
            {}
        </table>
    '''.format(
        ''.join('<th>{}</th>'.format(header) for header in headers),
        ''.join(table_rows)
    )
    output.clear_output(wait=True)
    with output:
        display(widgets.HTML(table_html))

# Define a function to handle button clicks
def on_button_click(button):
    snippet_index = snippets.index(button.description)
    update_table(snippet_index)

# Attach the on_button_click function to each button
for button in buttons:
    button.on_click(lambda btn: on_button_click(btn))

# Define a function to handle adding a new row
def on_add_click(button):
    text = text_input.value.strip()
    if text:
        color = color_picker.value
        snippet_id = str(uuid.uuid4())
        snippet_ids.append(snippet_id)
        snippets.append(text)
        array1_data = [1, 2, 3]  # Replace with your own logic
        array2_data = [text]
        array3_data = [True, False, True]  # Replace with your own logic
        added_data.append({
            'snippet_id': snippet_id,
            'arrays': [array1_data, array2_data, array3_data],
            'color': color
        })
        update_table(len(snippets) - 1)
        text_input.value = ""

# Attach the on_add_click function to the add button
add_button.on_click(on_add_click)

# Display the main box and initial table
display(main_box)
update_table(0)

IndexError: list index out of range

In [68]:
import ipywidgets as widgets
from IPython.display import display
import uuid

# Assume you have a list of text snippets and their associated arrays
selected_id = 0
snippets = ["hi", "bueno", "gracias", "mom"]
snippet_ids = [str(uuid.uuid4()) for _ in range(len(snippets))]
array1 = [[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12]]
array2 = [["a", "b", "c"], ["d", "e", "f"], ["g", "h", "i"], ["j", "k", "l"]]
array3 = [[True, False, True], [False, True, False], [True, True, False], [False, False, True]]
colors = ['lightblue', 'lightgreen', 'lightyellow', 'lightpink']

# Create a list of buttons for each text snippet
buttons = [widgets.Button(description=snippet, layout=widgets.Layout(width='auto', border='1px solid black', padding='0px', margin="0px")) for snippet in snippets]

# Create a box to hold the buttons
button_box = widgets.HBox(buttons, layout=widgets.Layout(display='flex', flex_flow='row wrap', align_items='stretch'))

# Create an output widget to display the table
output = widgets.Output()

# Create input widgets for adding new data
text_input = widgets.Text(description="Text:", margin="0px")
color_picker = widgets.ColorPicker(description="Color:", value='#ffffff', margin="0px")
add_button = widgets.Button(description="+")

# Create a box to hold the input widgets
input_box = widgets.VBox([text_input, color_picker, add_button])

# Create a box to hold the button_box and input_box
main_box = widgets.VBox([button_box, input_box, output])

# Define a function to update the table
def update_table():
    headers = ["Array 1", "Array 2", "Array 3"]
    table_rows = []
    
    if 0 <= selected_id < len(snippets):
        data = [array1[selected_id], array2[selected_id], array3[selected_id]]
        table_rows.extend(
            '<tr style="background-color: {};">{}</tr>'.format(
                colors[selected_id],
                ''.join('<td>{}</td>'.format(str(item)) for item in row)
            )
            for row in zip(*data)
        )
    
    table_html = '''
        <table>
            <tr>{}</tr>
            {}
        </table>
    '''.format(
        ''.join('<th>{}</th>'.format(header) for header in headers),
        ''.join(table_rows)
    )
    output.clear_output(wait=True)
    with output:
        display(widgets.HTML(table_html))

# Define a function to handle adding a new row
def on_add_click(button):
    global selected_id
    text = text_input.value.strip()
    if text:
        color = color_picker.value
        snippet_id = str(uuid.uuid4())
        snippet_ids.append(snippet_id)
        snippets.append(text)
        array1.append([1, 2, 3])  # Replace with your own logic
        array2.append([text])
        array3.append([True, False, True])  # Replace with your own logic
        colors.append(color)
        selected_id = len(snippets) - 1
        update_table()
        text_input.value = ""

# Attach the on_add_click function to the add button
add_button.on_click(on_add_click)

# Define a function to update the output and highlight the selected button
def on_button_click(button):
    global selected_id
    index = snippets.index(button.description)
    selected_id = index
    update_table()
    for i, btn in enumerate(buttons):
        btn.style.button_color = 'lightblue' if i == selected_id else 'transparent'

# Attach the on_button_click function to each button
for button in buttons:
    button.on_click(lambda btn: on_button_click(btn))

# Display the main box and initial table
display(main_box)
update_table()